This is a Vector-aware support assistant with tool calling

First we import and install dependencies

In [ ]:
!pip install openai python-dotenv --quiet

In [ ]:
import os
import json
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI

Environment setup

In [ ]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MODEL = "gpt-4"

add a light weight database

In [ ]:
def init_db():
    conn = sqlite3.connect("orders.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS orders (
        id INTEGER PRIMARY KEY,
        customer_name TEXT,
        status TEXT
    )
    """)

    cursor.execute("INSERT OR IGNORE INTO orders VALUES (1001, 'Alice', 'Shipped')")
    cursor.execute("INSERT OR IGNORE INTO orders VALUES (1002, 'Bob', 'Processing')")
    conn.commit()
    conn.close()

def get_order_status(order_id: int):
    conn = sqlite3.connect("orders.db")
    cursor = conn.cursor()
    cursor.execute("SELECT customer_name, status FROM orders WHERE id=?", (order_id,))
    result = cursor.fetchone()
    conn.close()

    if result:
        return {"order_id": order_id, "customer": result[0], "status": result[1]}
    return {"error": "Order not found"}

init_db()

add tool schema 

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_order_status",
            "description": "Retrieve order status by order ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "integer",
                        "description": "Order ID"
                    }
                },
                "required": ["order_id"]
            }
        }
    }
]

This next part is our chat runner

In [ ]:
def run_support_query(user_input):
    messages = [{"role": "user", "content": user_input}]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    message = response.choices[0].message

    if message.tool_calls:
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)

        result = get_order_status(arguments["order_id"])

        messages.append(message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)
        })

        final = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        return final.choices[0].message.content

    return message.content

In [ ]:
Run the code -test

In [ ]:
run_support_query("What is the status of order 1001?")